In [1]:
# torch imports
import torch
import torch._dynamo
torch._dynamo.config.suppress_errors = True
# Misc imports
import os 
import sys
import seaborn as sns
from pathlib import Path
sys.path.append('/storage/vbutoi/projects')
sys.path.append('/storage/vbutoi/libraries')
sys.path.append('/storage/vbutoi/projects/ESE')
sys.path.append('/storage/vbutoi/projects/UniverSegDev')
# Ionpy imports
from ionpy.util import Config
from ionpy.analysis import ResultsLoader

# Define some useful paths.
os.environ['DATAPATH'] = ':'.join((
       '/storage/vbutoi/datasets',
       '/storage'
))
# Set some defaults
rs = ResultsLoader()
sns.set_style("darkgrid")
torch.set_printoptions(linewidth=200)

# For using code without restarting.
%load_ext autoreload
%autoreload 2
# For using yaml configs.
%load_ext yamlmagic

In [ ]:
%%yaml default_cfg 

experiment:
    exp_root: '?'
    inference_seed: 40

log:
    root: '?'
    save_preds: False 
    log_interval: 10 
    log_pixel_stats: False 
    gether_inference_stats: True
    compute_global_metrics: False 

dataloader:
    batch_size: '?' 
    num_workers: '?' 
    pin_memory: True 

In [ ]:
%%yaml calibration_cfg 

local_calibration:
    num_prob_bins: 15
    neighborhood_width: 3

global_calibration:
    num_classes: 1 
    num_prob_bins: 15
    neighborhood_width: 3

In [ ]:
%%yaml model_cfg 

# For standard datasets
#####################################
model:
    pred_label: 0     
    _type: "standard"
    pretrained_exp_root : None
    checkpoint: 'min-val-abs_area_estimation_error'

## Gather Inference Options.

In [ ]:
%%yaml octa_cfg 

## Methods 
# name: "OCTA_Benchmark_Exps/Base"
# name: "OCTA_Benchmark_Exps/Optimal_Threshold_CrossEntropy"
# name: "OCTA_Benchmark_Exps/Optimal_Temperature_CrossEntropy"
name: "OCTA_Benchmark_Exps/LTS"

## Sweeps
# name: "OCTA_Benchmark_Exps/Threshold_Sweep_Dice"
# name: "OCTA_Benchmark_Exps/Threshold_Sweep_CrossEntropy"
# name: "OCTA_Benchmark_Exps/Temperature_Sweep_Dice"
# name: "OCTA_Benchmark_Exps/Temperature_Sweep_CrossEntropy"

base_model: 
    # - '/storage/vbutoi/scratch/ESE/training/August_2024/08_07_24_OCTA_FULLRES_SoftDice/20240807_144440-P7H9-459da42e97888bccb0aa79c297f388cd'
    # - '/storage/vbutoi/scratch/ESE/training/August_2024/08_07_24_OCTA_FULLRES_CrossEntropy/20240807_144627-M576-c3a81247e7e30c6bf47c72748764a898'
    - '/storage/vbutoi/scratch/ESE/calibration/August_2024/08_11_24_OCTA_FULLRES_LTS_LongRuns/20240811_123128-FZS0-5b5393971e116325f3cd4c334ad9b2c4'

In [ ]:
%%yaml experiment_cfg 

name: "OCTA_Benchmark_Exps/LTS"

# experiment: 
#     inf_kwargs:
        # threshold: 0.225
        #     - (0.00, ..., 0.25, 0.025)
        #     - (0.26, ..., 0.50, 0.025)
        #     - (0.51, ..., 0.75, 0.025)
        #     - (0.76, ..., 1.00, 0.025)
        # temperature: 1.06
        #     - (0.01, ..., 0.50, 0.025)
        #     - (0.51, ..., 1.00, 0.025)
        #     - (1.01, ..., 1.25, 0.025)
        #     - (1.26, ..., 1.50, 0.025)
        #     - (1.51, ..., 2.00, 0.025)
        #     - (2.01, ..., 2.50, 0.025)
        #     - (2.51, ..., 2.75, 0.025)
        #     - (2.76, ..., 3.00, 0.025)

base_model: '/storage/vbutoi/scratch/ESE/calibration/August_2024/08_11_24_OCTA_FULLRES_LTS_LongRuns/20240811_123128-FZS0-5b5393971e116325f3cd4c334ad9b2c4'

inference_data:
    split: 
        - ("cal", "val")

dataloader:
    batch_size: 1
    num_workers: 1

In [6]:
# Local imports
from ese.analysis.analysis_utils.submit_utils import get_ese_inference_configs

# Get the configs for the different runs.
base_cfg = Config(default_cfg).update([calibration_cfg, model_cfg])

# Get the different experiment cfg yamls.
updated_base_cfg, inf_cfgs = get_ese_inference_configs(
    exp_cfg=experiment_cfg, 
    base_cfg=base_cfg
)

In [ ]:
len(inf_cfgs)

## Running Jobs

In [ ]:
from ese.analysis.run_inference import get_cal_stats

In [ ]:
from ese.experiment import run_ese_exp

###### Run individual jobs
run_ese_exp(
    config=inf_cfgs[0], 
    job_func=get_cal_stats,
    run_name='debug',
    show_examples=True,
    gpu='0',
)

In [ ]:
# from ese.experiment import submit_ese_exps 

# #### Run Batch Jobs
# submit_ese_exps(
#     group="inference",
#     base_cfg=updated_base_cfg,
#     exp_cfg=experiment_cfg,
#     config_list=inf_cfgs,
#     job_func=get_cal_stats,
#     available_gpus=['1', '2', '3']
# )